In [6]:
import kagglehub
import pandas as pd
import os
import cv2

In [4]:
path = kagglehub.dataset_download("stanislavlevendeev/hazmat-detection")
video_directory = os.environ["PATH_TO_DATA"]
print("Path to dataset files:", path)
print("Path to video files:", video_directory)

Path to dataset files: C:\Users\stani\.cache\kagglehub\datasets\stanislavlevendeev\hazmat-detection\versions\4
Path to video files: C:/Users/stani/Documents/WagonVideos


In [23]:
df = pd.read_csv(path + "/labels_dataframe.csv")
videos = df["Source"].unique()
videos

array(['1690279852.mp4', '1690281365.mp4', '1692830440.mp4',
       '1690801380.mp4', '1691487366.mp4', '1692787289.mp4',
       '1692875102.mp4', '1692945482.mp4', '1693811855.mp4',
       '1693954819.mp4', '1693805101.mp4', '1693820172.mp4',
       '1691496786.mp4', '1692872075.mp4', '1693308534.mp4',
       '1693308657.mp4', '1693309263.mp4', '1693820241.mp4',
       '1693820504.mp4', '1693820904.mp4', '1696009577.mp4',
       '1696374314.mp4', '1696416413.mp4', '1696441496.mp4'], dtype=object)

In [30]:

for row in videos:
    video_path = video_directory + '/' + row
    
    if (os.path.exists(video_path) == False):
        print(f"File {video_path} not found")
        continue
    processed_source = row.split('.')[0]
    print(f"File {video_path} found")
    # Open video file
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"File {video_path} has {frame_count} frames")
    lables = df[df['Source'] == row]
    # save each frame and .txt file with labels
    frame_idx = 0
    while frame_idx < frame_count:
        ret, frame = cap.read()  # Read each frame
        if not ret:
            break  # End of video
        if(frame_idx in lables['Relative Frame'].values):
            # Save frame
            frame_index = str(frame_idx).zfill(5)
            frame_name = f"{processed_source}_{frame_index}.jpg"
            cv2.imwrite(path + "/yolo/frames/" + frame_name, frame)
            # Save labels
            label_name = f"{processed_source}_{frame_index}.txt"
            with open(path.replace('/','\\') + "\\yolo\\labels\\" + label_name, "w") as f:
                row = lables[lables['Relative Frame'] == frame_idx]
                f.write(f"1 {row['XTL'].values[0]} {row['YTL'].values[0]} {row['XBR'].values[0]} {row['YBR'].values[0]}\n")
        else:
             # Save frame
            frame_index = str(frame_idx).zfill(5)
            frame_name = f"{processed_source}_{frame_index}.jpg"
            cv2.imwrite(path + "/yolo/frames/" + frame_name, frame)
            # Save labels
            label_name = f"{processed_source}_{frame_index}.txt"
            with open(path.replace('/','\\') + "\\yolo\\labels\\" + label_name, "w") as f:
                f.write(f"")
        frame_idx += 1
    cap.release()

File C:/Users/stani/Documents/WagonVideos/1690279852.mp4 not found
File C:/Users/stani/Documents/WagonVideos/1690281365.mp4 found
File C:/Users/stani/Documents/WagonVideos/1690281365.mp4 has 733 frames
File C:/Users/stani/Documents/WagonVideos/1692830440.mp4 not found
File C:/Users/stani/Documents/WagonVideos/1690801380.mp4 not found
File C:/Users/stani/Documents/WagonVideos/1691487366.mp4 not found
File C:/Users/stani/Documents/WagonVideos/1692787289.mp4 not found
File C:/Users/stani/Documents/WagonVideos/1692875102.mp4 not found
File C:/Users/stani/Documents/WagonVideos/1692945482.mp4 not found
File C:/Users/stani/Documents/WagonVideos/1693811855.mp4 not found
File C:/Users/stani/Documents/WagonVideos/1693954819.mp4 not found
File C:/Users/stani/Documents/WagonVideos/1693805101.mp4 not found
File C:/Users/stani/Documents/WagonVideos/1693820172.mp4 not found
File C:/Users/stani/Documents/WagonVideos/1691496786.mp4 not found
File C:/Users/stani/Documents/WagonVideos/1692872075.mp4 not 